# Policy Improvement : $\pi = greedy(V)$ 

## Notion d'improvement

Pour avoir une meilleure intuition à propos de cet algorithme, il est d'abord intéressant de comparer les policies entre elles. Comment, à partir de deux policies, déterminer la meilleure ?

Il faut pour cela définir une manière de les classer. Soit deux policies déterministiques $\pi$ et $\pi'$.


Si &emsp; &ensp; $q_\pi(s, \pi'(s)) \geq q_\pi(s, \pi(s))$ &ensp; $\forall s$, </br>
alors &ensp; $v_{\pi'}(s) \geq v_\pi(s)$ &ensp; $\forall s$ </br> 
&ensp; &ensp; &ensp; &ensp; $\pi' \geq \pi$

La condition $q_\pi(s, \pi'(s)) \geq q_\pi(s, \pi(s))$ $\forall s$ requiert que, pour tout state $s$, suivre $\pi'$ pour une timestep et ensuite suivre $\pi$ rapporte au moins autant, voir plus, de cumulative reward que suivre $\pi$ dès le state $s$. Si cette condition est respectée, alors il est raisonnable de penser que $\pi'$ est, au pire, aussi bonne que $\pi$. Cela se traduit par l'implication $v_{\pi'}(s) \geq v_\pi(s) \forall s$.

Maintenant, définissons une policy déterministique $\pi$ , que nous allons improve. Pour cela, attribuons à la policy $\pi'$ l'action $a$ qui maximise $q_\pi(s,a)$, pour tout state $s$.

\begin{align}
\large \pi'(s) \large \leftarrow argmax_a \; q_\pi(s, a) \quad \forall s
\end{align}

Ainsi, pour chaque state $s$, la policy $\pi'$ se voit attribuer la meilleure action à exécuter. Si $\pi'(s) = \pi(s)$, alors il n'y a pas eu d'improvement car l'action attribuée à $\pi'$ était déjà celle de $\pi$. Si toutefois $\pi'(s) \ne \pi(s)$, alors l'action attribuée à $\pi'$ est par définition meilleure (i.e. l'expected cumulative reward sera plus grand) que l'action $\pi(s)$.

Ainsi, on peut affirmer que:

\begin{align}
\large q_\pi(s, \pi(s')) \large \geq q_\pi(s, \pi(s)) \quad \forall s
\end{align}

Cela prouve donc que la policy $\pi'$ est au pire aussi bonne que $\pi$.

Les policies utilisées ici pour démontrer l'improvement sont déterministiques pour rendre la preuve plus simple, mais cela s'applique également aux policies stochastiques.

## L'algorithme

L'algorithme de policy improvement consiste à améliorer une policy $\pi$ par rapport à sa state-value function $Q$, pour chaque state $s$. Il est souvent dit de cet improvement que la policy agit "greedily" par rapport à la state-value function.

L'improvement assigne à $\pi$ la meilleure action $a$ pour chaque state $s$. Chaque action est évaluée grâce à la value function $V$, connue. (calculée par l'algorithme de policy evaluation)

Il peut ainsi être décrit comme suit pour une policy $\pi$ déterministique:

\begin{align}
\large \pi(s) \; & \large \leftarrow argmax_a \; Q(s, a) \quad \forall s
\\
& \large \leftarrow argmax_a \; [\sum_{s'} p(s'|s,a) [R_{ss'}^a + γ  V(s')]] \quad \forall s
\end{align}

Ainsi, pour chaque state $s$, l'algorithme devra calculer la value $\sum_{s'} p(s'|s,a) [R_{ss'}^a + γ  v_\pi(s')]$ de chaque action $a$, et assignera à la policy $\pi$ l'action qui a la plus grande value.

## Exemple Gridworld

In [1]:
import numpy as np

import envs.gridworld_dennybritz as grd

In [2]:
env = grd.GridworldEnv()

### Helper function

#### Cette fonction sert d'interface avec l'environnement.
* `compute_q_value_for_s_a(env, V, s, a, gamma)` retourne, pour le state $s$ et l'action $a$ spécifiés la valeur $Q(s,a)$ soit $\sum_{s'} p(s'|s,a) [R_{ss'}^a + \gamma V(s')]$. <br>Cette fonction interroge la fonction `P[s][a]` de l'environnement qui renvoit une liste de tuple de la forme : `(p(s'|s,a), s', r(s,a,s'), done?)`. L'algorithme loop sur ces tuples et ajoute à `q` (équivalent à $Q(s,a)$) la valeur $p(s'|s,a) [R_{ss'}^a + \gamma V(s')]$.

In [3]:
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0
    
    for p_sPrime, sPrime, r_ss_a, done in env.P[s][a]:
        q += p_sPrime * (r_ss_a + gamma * V[sPrime])
        
    return q

### Initialization

La policy $\pi$ à improve est initalizée, donnant à chacune des 4 actions une probabilité de $0.25$ à être exécutée, dans tout state $s$. $\pi$ est ici une table de dimensions $|S|x|A|$. Ainsi, la probabilité d'exécuter $a$ dans le state $s$ est stored dans `pi[s][a]`.

La value function $V$ est celle calculée par l'algorithme d'évaluation.

L'hyperparamètre $\gamma$ est également initializé. Il prend la valeur de 1, cela ne posera pas de problème car l'horizon est dans ce cas là fini.

In [4]:
pi = np.ones([env.nS, env.nA]) * 0.25
gamma = 1

V = np.array([[0], [-14], [-20], [-22], [-14], [-18], [-20], [-20], [-20], [-20], [-18], [-14], [-22], [-20], [-14], [0]])

### Improvement

Pour chaque state $s$, les entrées de la table `q_s` de dimensions $|A|x|1|$ sont toutes initializées à 0. L'algorithme doit alors remplir cette table avec la value $Q(s,a)$ = $\sum_{s'} p(s'|s,a) [R_{ss'}^a + γ V(s')]$ pour chaque action $a$.

Ce calcul est réalisé par la fonction `compute_q_value_for_s_a`.

Une fois $Q(s, a)$ calculé pour tout $a$, l'algorithme assigne une probabilité de $1$ à l'action $a$ qui maximize $Q(s, a)$, et une probabilité de $0$ à toutes les autres actions.

In [5]:
for s in range(env.nS):
    q_s = np.zeros([env.nA, 1])
    
    for a in range(env.nA):        
        q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)
    
    best_a = np.argmax(q_s)
    pi[s] = np.eye(env.nA)[best_a]

In [6]:
pi

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

La policy $\pi$ obtenue est représenté dans le gridworld ci-dessous, chaque flèche correspondant à l'action choisie par $\pi$.

<img src="img/gridworld_actions.png" width="250" height="250" align="center"/>

Dans ce cas-là, il est facile de déduire cette policy depuis la value function fournie, sans appliquer l'algorithme décrit:

<img src="img/gridworld_states_values.png" width="250" height="250" align="center"/>

Il se trouve que dans ce cas-là, il n'a suffit que d'une évaluation ($V$) et d'un improvement pour que la policy $\pi$ trouvée soit optimale, elle est donc égale à $\pi^*$. Dans des cas plus complexes, avec notamment plus de states, il faut plusieurs évaluations et improvements pour trouver $\pi^*$, c'est le principe même de l'algorithme policy iteration.